In [2]:
import pandas as pd
import numpy as np

In [7]:
#Load the dataset
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
rating=pd.read_csv('ml-100k/u.data',sep='\t',names=r_cols,encoding='latin-1')

In [9]:
rating.head(2)

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [15]:
#to cehck no of rows 
rating.user_id.unique().shape[0]

943

In [16]:
rating.movie_id.unique().shape[0]#no of movies

1682

In [22]:
rating.shape

(100000, 4)

In [20]:
mean_user_rating=rating.mean(axis=1)

In [21]:
mean_user_rating

0        2.203128e+08
1        2.229296e+08
2        2.197219e+08
3        2.201518e+08
4        2.215995e+08
             ...     
99995    2.200442e+08
99996    2.199491e+08
99997    2.186993e+08
99998    2.205998e+08
99999    2.199900e+08
Length: 100000, dtype: float64

# Create pivot table for user and movie basedon ratings

In [25]:
data_matrix=rating.pivot_table(index='user_id',columns='movie_id',values='rating')
data_matrix.head(4)

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
#Replace null value with 0
data_matrix=data_matrix.replace(np.nan,0)

In [42]:
data_matrix.shape

(943, 1682)

# Find Cosine Similatity for user and Item

In [31]:
from sklearn.metrics.pairwise import pairwise_distances 
user_similarity = pairwise_distances(data_matrix, metric='cosine')

In [32]:
user_similarity

array([[1.33226763e-15, 8.33069016e-01, 9.52540457e-01, ...,
        8.51383057e-01, 8.20492117e-01, 6.01825261e-01],
       [8.33069016e-01, 0.00000000e+00, 8.89408675e-01, ...,
        8.38515222e-01, 8.27732187e-01, 8.94202122e-01],
       [9.52540457e-01, 8.89408675e-01, 0.00000000e+00, ...,
        8.98757435e-01, 8.66583851e-01, 9.73444131e-01],
       ...,
       [8.51383057e-01, 8.38515222e-01, 8.98757435e-01, ...,
        0.00000000e+00, 8.98358201e-01, 9.04880419e-01],
       [8.20492117e-01, 8.27732187e-01, 8.66583851e-01, ...,
        8.98358201e-01, 0.00000000e+00, 8.17535338e-01],
       [6.01825261e-01, 8.94202122e-01, 9.73444131e-01, ...,
        9.04880419e-01, 8.17535338e-01, 0.00000000e+00]])

# Using formula for user and item we are calcuating the score value

In [36]:
def predict(rating,similarity,type='user'):
    if type=='user':
        #Calculate mean rating of each user
        mean_rating=rating.mean(axis=1).to_numpy()
        #Compute diff between mean rating and userrating to normalize the data
        # using np.newaxis so than mean rating has same format as rating
        rating_diff=(rating-mean_rating[:,np.newaxis])
        #Formula Compute weighed sum of normalised rating
        #Divide sum of absolute similiarity for each user for normalize the prediction
        #Predicted rating is adjusting to original scale by adding uer rating
        pred=mean_rating[:,np.newaxis]+similarity.dot(rating_diff)/np.array([np.abs(similarity).sum(axis=1)])
    elif type=='item':
        pred=rating.dot(similarity)/np.array([np.abs(similarity).sum(axis=1)])
    return pred



In [43]:
user_prediction=predict(data_matrix,user_similarity,type='user')

In [44]:
user_prediction

array([[ 2.06532606,  0.73430275,  0.62992381, ...,  0.39359041,
         0.39304874,  0.3927712 ],
       [ 1.76308836,  0.38404019,  0.19617889, ..., -0.08837789,
        -0.0869183 , -0.08671183],
       [ 1.79590398,  0.32904733,  0.15882885, ..., -0.13699223,
        -0.13496852, -0.13476488],
       ...,
       [ 1.59151513,  0.27526889,  0.10219534, ..., -0.16735162,
        -0.16657451, -0.16641377],
       [ 1.81036267,  0.40479877,  0.27545013, ..., -0.00907358,
        -0.00846587, -0.00804858],
       [ 1.8384313 ,  0.47964837,  0.38496292, ...,  0.14686675,
         0.14629808,  0.14641455]])

### As per User based filtering ,first have to find similarity between the input user and others

In [62]:
inp_user=34

In [63]:
#2. Convert the user_sim table into DataFrame
user_sim_table=pd.DataFrame(user_similarity)
user_sim_table.head(2)

,0,1,2,3,4,5,6,7,8,9,...,933,934,935,936,937,938,939,940,941,942
0,1.332268e-15,0.833069,0.952540,0.935642,0.621525,0.569761,0.559633,0.680928,0.921862,0.623456,...,0.630473,0.880518,0.725124,0.810295,0.802674,0.881905,0.685928,0.851383,0.820492,0.601825
1,8.330690e-01,0.000000,0.889409,0.821879,0.927021,0.754157,0.892672,0.896656,0.838952,0.840138,...,0.843014,0.692058,0.641211,0.575954,0.680111,0.771417,0.773210,0.838515,0.827732,0.894202


In [64]:
#3. Find similarity user for 24 using cosine table

sim_input_user=user_sim_table[inp_user].sort_values(ascending=True).head(5).index

In [65]:
sim_input_user

Index([34, 450, 852, 811, 509], dtype='int64')

In [66]:
#4.Convert in to list
sim_input_user_lst=list(sim_input_user)

In [57]:
sim_input_user_lst[0]

24

In [67]:
#5. Using similar_user_input,can select movie id from ratings table
rating[rating['user_id']==sim_input_user_lst[0]]['rating']

1530     4
2600     5
2859     4
4025     4
4225     2
9465     5
14081    4
16582    5
18163    5
19374    5
23291    5
25581    5
28170    1
31084    5
53184    5
81341    1
89679    5
92131    5
94521    2
99001    4
Name: rating, dtype: int64

In [70]:
sim_user_movieid_lst=[]
for sim_user in sim_input_user_lst:
    sim=list(rating[rating['user_id']==sim_user]['movie_id'])
    sim_user_movieid_lst.append(sim)

In [71]:
sim_user_movieid_lst

[[312,
  242,
  690,
  310,
  259,
  299,
  245,
  332,
  329,
  286,
  1024,
  324,
  294,
  292,
  990,
  289,
  898,
  899,
  288,
  991],
 [470,
  783,
  1147,
  100,
  58,
  142,
  801,
  216,
  785,
  162,
  144,
  462,
  384,
  739,
  1269,
  179,
  38,
  659,
  252,
  1160,
  566,
  496,
  679,
  387,
  908,
  265,
  1226,
  183,
  1037,
  194,
  902,
  1263,
  64,
  1036,
  942,
  467,
  1212,
  97,
  762,
  143,
  583,
  480,
  846,
  561,
  487,
  2,
  237,
  39,
  778,
  815,
  708,
  400,
  495,
  382,
  518,
  110,
  1203,
  710,
  660,
  152,
  304,
  520,
  604,
  642,
  494,
  211,
  847,
  559,
  1446,
  204,
  904,
  1092,
  281,
  526,
  139,
  200,
  367,
  553,
  523,
  685,
  478,
  177,
  742,
  1044,
  509,
  89,
  238,
  29,
  15,
  837,
  65,
  49,
  176,
  434,
  729,
  1184,
  70,
  728,
  174,
  1047,
  723,
  477,
  419,
  689,
  582,
  270,
  7,
  155,
  12,
  774,
  383,
  488,
  145,
  401,
  301,
  468,
  157,
  771,
  650,
  288,
  95,
  1,
  154,
  

In [73]:
#6. Convert all the list as single
import itertools
similar_user_movieid_single_list=list(itertools.chain.from_iterable(sim_user_movieid_lst))

In [74]:
Unique_movieid_similar_user=set(similar_user_movieid_single_list)

In [75]:
#Input user watched movie_list
input_user_watched=rating[rating['user_id']==inp_user]['movie_id']

In [76]:
input_user_watched

1530      312
2600      242
2859      690
4025      310
4225      259
9465      299
14081     245
16582     332
18163     329
19374     286
23291    1024
25581     324
28170     294
31084     292
53184     990
81341     289
89679     898
92131     899
94521     288
99001     991
Name: movie_id, dtype: int64

In [77]:
#9. Create a list which should have recom movieid to the input user

recom_list=[]
for user_id in Unique_movieid_similar_user:
    if user_id in input_user_watched:
        pass
    else:
        recom_list.append(user_id)

In [78]:
#Recommended movie id
recom_list

[1,
 2,
 3,
 4,
 7,
 10,
 11,
 12,
 13,
 15,
 22,
 23,
 25,
 26,
 28,
 29,
 33,
 35,
 38,
 39,
 43,
 44,
 47,
 49,
 50,
 51,
 54,
 56,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 109,
 110,
 111,
 112,
 114,
 117,
 118,
 121,
 122,
 123,
 125,
 126,
 127,
 131,
 132,
 133,
 134,
 135,
 136,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 207,
 208,
 210,
 211,
 213,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 241,
 242,
 243,
 245,
 250,
 252,
 254,
 257,
 258,
 259,
 260,
 264,
 265,
 266,
 268,

In [80]:
sorted(recom_list)

[1,
 2,
 3,
 4,
 7,
 10,
 11,
 12,
 13,
 15,
 22,
 23,
 25,
 26,
 28,
 29,
 33,
 35,
 38,
 39,
 43,
 44,
 47,
 49,
 50,
 51,
 54,
 56,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 109,
 110,
 111,
 112,
 114,
 117,
 118,
 121,
 122,
 123,
 125,
 126,
 127,
 131,
 132,
 133,
 134,
 135,
 136,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 207,
 208,
 210,
 211,
 213,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 241,
 242,
 243,
 245,
 250,
 252,
 254,
 257,
 258,
 259,
 260,
 264,
 265,
 266,
 268,

In [81]:
sorted(Unique_movieid_similar_user)

[1,
 2,
 3,
 4,
 7,
 10,
 11,
 12,
 13,
 15,
 22,
 23,
 25,
 26,
 28,
 29,
 33,
 35,
 38,
 39,
 43,
 44,
 47,
 49,
 50,
 51,
 54,
 56,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 109,
 110,
 111,
 112,
 114,
 117,
 118,
 121,
 122,
 123,
 125,
 126,
 127,
 131,
 132,
 133,
 134,
 135,
 136,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 207,
 208,
 210,
 211,
 213,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 241,
 242,
 243,
 245,
 250,
 252,
 254,
 257,
 258,
 259,
 260,
 264,
 265,
 266,
 268,

In [82]:
#Checking the common movie list

In [84]:
list(set(Unique_movieid_similar_user)&set(input_user_watched))

[1024,
 898,
 259,
 899,
 286,
 288,
 289,
 292,
 294,
 299,
 690,
 310,
 312,
 324,
 329,
 332,
 990,
 991,
 242,
 245]

In [85]:
#Create dataframe for prediction
user_pred=pd.DataFrame(user_prediction)

In [133]:
user_pred

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
0,2.065326,0.734303,0.629924,1.010669,0.640686,0.476150,1.784569,1.163032,1.513350,0.704478,...,0.394041,0.394434,0.393981,0.392972,0.393344,0.392272,0.394909,0.393590,0.393049,0.392771
1,1.763088,0.384040,0.196179,0.731538,0.225643,0.003892,1.493597,0.876153,1.108467,0.261991,...,-0.086942,-0.085491,-0.087137,-0.088158,-0.087298,-0.089288,-0.087468,-0.088378,-0.086918,-0.086712
2,1.795904,0.329047,0.158829,0.684154,0.173277,-0.035621,1.488230,0.835769,1.135426,0.236383,...,-0.134795,-0.133537,-0.135543,-0.136438,-0.135041,-0.137611,-0.136374,-0.136992,-0.134969,-0.134765
3,1.729951,0.293913,0.127741,0.644932,0.142143,-0.062261,1.437010,0.796249,1.096663,0.211789,...,-0.161413,-0.160220,-0.161542,-0.162586,-0.161634,-0.163877,-0.162283,-0.163080,-0.161442,-0.161248
4,1.796651,0.454474,0.354422,0.763130,0.359539,0.195987,1.547370,0.908904,1.292027,0.437954,...,0.101762,0.102405,0.101923,0.100839,0.101711,0.099951,0.102515,0.101233,0.101075,0.101201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,1.676950,0.346339,0.177518,0.689906,0.199740,0.003297,1.429565,0.830905,1.070986,0.262183,...,-0.092434,-0.091197,-0.092851,-0.093801,-0.092953,-0.094539,-0.092217,-0.093378,-0.092686,-0.092423
939,1.822346,0.419125,0.286430,0.715605,0.294442,0.106633,1.514591,0.853050,1.195304,0.359260,...,0.014060,0.014688,0.014123,0.013060,0.013669,0.011978,0.014065,0.013021,0.013639,0.013796
940,1.591515,0.275269,0.102195,0.624383,0.133762,-0.069553,1.320734,0.765529,1.035088,0.192697,...,-0.166179,-0.164981,-0.166278,-0.167392,-0.166679,-0.168486,-0.166217,-0.167352,-0.166575,-0.166414
941,1.810363,0.404799,0.275450,0.726616,0.281316,0.087068,1.550310,0.850057,1.205745,0.342987,...,-0.008362,-0.007757,-0.008225,-0.009218,-0.008232,-0.010138,-0.008009,-0.009074,-0.008466,-0.008049


In [134]:
user_pred[inp_user]#its predicting the user but we need movie id to predict

0      0.413512
1     -0.061613
2     -0.111001
3     -0.138049
4      0.119991
         ...   
938   -0.070660
939    0.036352
940   -0.143268
941    0.013372
942    0.166584
Name: 34, Length: 943, dtype: float64

In [88]:
user_pred_trans=user_pred.T

In [93]:
pred_input_user=user_pred_trans[34]#predicting the movie id for this input user

In [109]:
pred_input_user

0       1.740469
1       0.283366
2       0.114987
3       0.645096
4       0.127844
          ...   
1677   -0.183977
1678   -0.182367
1679   -0.183172
1680   -0.181400
1681   -0.181306
Name: 34, Length: 1682, dtype: float64

In [94]:
pre_input_user=pd.DataFrame(pred_input_user)


In [108]:
pre_input_user

,34
0,1.740469
1,0.283366
2,0.114987
3,0.645096
4,0.127844
...,...
1677,-0.183977
1678,-0.182367
1679,-0.183172
1680,-0.181400


In [96]:
pre_input_user.head(3)

,34
0,1.740469
1,0.283366
2,0.114987


In [97]:
#Transpose for checking the movie id for this specific user
pre_input_user.T

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
34,1.740469,0.283366,0.114987,0.645096,0.127844,-0.081329,1.448638,0.794313,1.08901,0.194232,...,-0.181256,-0.180156,-0.181837,-0.182819,-0.181432,-0.183977,-0.182367,-0.183172,-0.1814,-0.181306


In [100]:
inp_user_pred=pd.DataFrame(user_pred_trans[inp_user])

In [101]:
inp_user_pred_T=inp_user_pred.T

In [106]:
inp_user_pred_T

,0,1,2,3,4,5,6,7,8,9,...,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681
34,1.740469,0.283366,0.114987,0.645096,0.127844,-0.081329,1.448638,0.794313,1.08901,0.194232,...,-0.181256,-0.180156,-0.181837,-0.182819,-0.181432,-0.183977,-0.182367,-0.183172,-0.1814,-0.181306


In [105]:
recom_list

[1,
 2,
 3,
 4,
 7,
 10,
 11,
 12,
 13,
 15,
 22,
 23,
 25,
 26,
 28,
 29,
 33,
 35,
 38,
 39,
 43,
 44,
 47,
 49,
 50,
 51,
 54,
 56,
 58,
 59,
 60,
 61,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 109,
 110,
 111,
 112,
 114,
 117,
 118,
 121,
 122,
 123,
 125,
 126,
 127,
 131,
 132,
 133,
 134,
 135,
 136,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 151,
 152,
 153,
 154,
 155,
 157,
 158,
 161,
 162,
 163,
 164,
 166,
 167,
 168,
 169,
 170,
 172,
 173,
 174,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 199,
 200,
 202,
 203,
 204,
 205,
 207,
 208,
 210,
 211,
 213,
 214,
 215,
 216,
 218,
 220,
 221,
 222,
 223,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 237,
 238,
 239,
 241,
 242,
 243,
 245,
 250,
 252,
 254,
 257,
 258,
 259,
 260,
 264,
 265,
 266,
 268,

In [121]:
inp_user_pred_T[895].values

array([-0.00981091])

In [122]:
# From recomd list select hightest rated film which would like by the user. Based on User prediction
highes_rated_list=[]
for re in recom_list:
    value=inp_user_pred_T[re].values
    if value>=1:
        highes_rated_list.append(re)

In [124]:
highes_rated_list

[11,
 49,
 63,
 78,
 88,
 95,
 97,
 99,
 126,
 167,
 172,
 173,
 180,
 182,
 190,
 194,
 203,
 215,
 221,
 257,
 268,
 275,
 287,
 299,
 301,
 312,
 356,
 422,
 482]

In [111]:

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
encoding='latin-1')

In [113]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [125]:
#Creating Movie List based on recom movieid
movie_title=[]
for movieid in highes_rated_list:
    mov=items[items['movie id']==movieid]['movie title'].values
    movie_title.append(mov)

In [126]:
movie_title

[array(['Seven (Se7en) (1995)'], dtype=object),
 array(['I.Q. (1994)'], dtype=object),
 array(['Santa Clause, The (1994)'], dtype=object),
 array(['Free Willy (1993)'], dtype=object),
 array(['Sleepless in Seattle (1993)'], dtype=object),
 array(['Aladdin (1992)'], dtype=object),
 array(['Dances with Wolves (1990)'], dtype=object),
 array(['Snow White and the Seven Dwarfs (1937)'], dtype=object),
 array(['Spitfire Grill, The (1996)'], dtype=object),
 array(['Private Benjamin (1980)'], dtype=object),
 array(['Empire Strikes Back, The (1980)'], dtype=object),
 array(['Princess Bride, The (1987)'], dtype=object),
 array(['Apocalypse Now (1979)'], dtype=object),
 array(['GoodFellas (1990)'], dtype=object),
 array(['Henry V (1989)'], dtype=object),
 array(['Sting, The (1973)'], dtype=object),
 array(['Unforgiven (1992)'], dtype=object),
 array(['Field of Dreams (1989)'], dtype=object),
 array(['Breaking the Waves (1996)'], dtype=object),
 array(['Men in Black (1997)'], dtype=object),
 array

In [127]:
#to make it as list
movie_title_lst=[]
for movie_lst in movie_title:
    mv=list(movie_lst)
    movie_title_lst.append(mv)

In [128]:
movie_title_lst

[['Seven (Se7en) (1995)'],
 ['I.Q. (1994)'],
 ['Santa Clause, The (1994)'],
 ['Free Willy (1993)'],
 ['Sleepless in Seattle (1993)'],
 ['Aladdin (1992)'],
 ['Dances with Wolves (1990)'],
 ['Snow White and the Seven Dwarfs (1937)'],
 ['Spitfire Grill, The (1996)'],
 ['Private Benjamin (1980)'],
 ['Empire Strikes Back, The (1980)'],
 ['Princess Bride, The (1987)'],
 ['Apocalypse Now (1979)'],
 ['GoodFellas (1990)'],
 ['Henry V (1989)'],
 ['Sting, The (1973)'],
 ['Unforgiven (1992)'],
 ['Field of Dreams (1989)'],
 ['Breaking the Waves (1996)'],
 ['Men in Black (1997)'],
 ['Chasing Amy (1997)'],
 ['Sense and Sensibility (1995)'],
 ["Marvin's Room (1996)"],
 ['Hoodlum (1997)'],
 ['In & Out (1997)'],
 ['Midnight in the Garden of Good and Evil (1997)'],
 ['Client, The (1994)'],
 ['Aladdin and the King of Thieves (1996)'],
 ['Some Like It Hot (1959)']]

In [130]:
import itertools
Final_Recommend_movie=list(itertools.chain.from_iterable(movie_title_lst))

In [131]:
Final_Recommend_movie

['Seven (Se7en) (1995)',
 'I.Q. (1994)',
 'Santa Clause, The (1994)',
 'Free Willy (1993)',
 'Sleepless in Seattle (1993)',
 'Aladdin (1992)',
 'Dances with Wolves (1990)',
 'Snow White and the Seven Dwarfs (1937)',
 'Spitfire Grill, The (1996)',
 'Private Benjamin (1980)',
 'Empire Strikes Back, The (1980)',
 'Princess Bride, The (1987)',
 'Apocalypse Now (1979)',
 'GoodFellas (1990)',
 'Henry V (1989)',
 'Sting, The (1973)',
 'Unforgiven (1992)',
 'Field of Dreams (1989)',
 'Breaking the Waves (1996)',
 'Men in Black (1997)',
 'Chasing Amy (1997)',
 'Sense and Sensibility (1995)',
 "Marvin's Room (1996)",
 'Hoodlum (1997)',
 'In & Out (1997)',
 'Midnight in the Garden of Good and Evil (1997)',
 'Client, The (1994)',
 'Aladdin and the King of Thieves (1996)',
 'Some Like It Hot (1959)']

In [132]:
def userbased(input_user,user_similarity,user_predictions,similar_user_count,thres):
    #Convert the user_sim table into DataFrame
    user_sim_table=pd.DataFrame(user_similarity)
    #Find similarity user for 78 using cosine table
    similar_input_user= user_sim_table[input_user].sort_values(ascending=True).head(similar_user_count).index
    #Convert in to list
    similar_user_input=list(similar_input_user) 
    #Using similar_user_input,can select movie id from ratings table
    similar_user_movieid_list=[]
    for sim_user in similar_user_input:
        sim=list(ratings[ratings['user_id']==sim_user]['movie_id'])
        similar_user_movieid_list.append(sim)
    #Converting as a whole list
    import itertools
    similar_user_movieid_single_list=list(itertools.chain.from_iterable(similar_user_movieid_list))
    #Unique movieid from the list
    Unique_movieid_similar_user=set(similar_user_movieid_single_list)
    #Input user watched movie_list
    input_user_watched_movieid=list(ratings[ratings['user_id']==input_user]['movie_id'].values)
    #Create a list which should have recom movieid to the input user
    recom=[]
    for per_id in Unique_movieid_similar_user:
        if(per_id in input_user_watched_movieid):
            pass
        else:
            recom.append(per_id)
    #From recommendation list selecting only hightest rated(predicted) value
    highest_Rated=[]
    user_pred=pd.DataFrame(user_prediction)
    user_pred_Trans=user_pred.T
    input_user_pre=pd.DataFrame(user_pred_Trans[input_user])
    input_user_pred=input_user_pre.T
    for re in recom:
        value=input_user_pred[re].values
        if(value>=thres):
            highest_Rated.append(re)
    i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
    'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
    'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,encoding='latin-1')
    #Creating Movie List based on recom movieid
    movie_title=[]
    for movieid in highest_Rated:
        mov=items[items['movie id']==movieid]['movie title'].values
        movie_title.append(mov)
    #Converting into pure list
    movie_title_list=[]
    for m in movie_title:
        print(m)
        mv=list(m)
        movie_title_list.append(mv)
    #Converting into whole list
    import itertools
    Final_Recommend_movie=list(itertools.chain.from_iterable(movie_title_list))
    print("The common Movie in Recom & User:",list(set(recom)&set(input_user_watched_movieid)))
    return Final_Recommend_movie
    